In [ ]:
# Imports
import tensorflow as tf
import tensorflow_datasets as tfds

import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm
import numpy as np
# Set batch size for training and validation
batch_size = 32

In [ ]:
# List all available models
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}

In [ ]:
model_dictionary

In [ ]:
models_to_apply = {}
def addModels():
    try:
        modelsToAdd = input("What would you like to add? ")
        models_to_apply[modelsToAdd]=model_dictionary[modelsToAdd]
    except KeyError:
        print("No such model")

In [ ]:
addModels()

In [ ]:
models_to_apply

In [ ]:
img_height=224
img_width=224

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir='/content/drive/MyDrive/output/train'
val_dir='/content/drive/MyDrive/output/val'
test_dir='/content/drive/MyDrive/output/test'

In [ ]:
# Load the normalized images
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.15)
#validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
# Change the batchsize according to your system RAM
train_batchsize = 32
val_batchsize = 32
test_batchsize=2
# Data generator for training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224,224),
        batch_size=train_batchsize,
        class_mode='categorical',
        subset='training')

# Data generator for validation data
validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224,224),
        batch_size=val_batchsize,
        class_mode='categorical',
        subset='validation',
        shuffle=False)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224,224),
        batch_size=test_batchsize,
        class_mode='categorical',
        shuffle=False)

In [ ]:
from keras.callbacks import TensorBoard
%load_ext tensorboard
import tensorflow as tf
import datetime

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:

# Loop over each model available in Keras
#model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': []}
for model_name, model in tqdm(models_to_apply.items()):
    # load the pre-trained model with global average pooling as the last layer and freeze the model weights
    input_shape=(224,224,3)
    pre_trained_model = model(include_top=False, pooling='avg', input_shape=input_shape)
    pre_trained_model.trainable = False

    # custom modifications on top of pre-trained model
    clf_model = tf.keras.models.Sequential()
    clf_model.add(pre_trained_model)
    clf_model.add(tf.keras.layers.Dense(11, activation='softmax'))
    clf_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
    #history = clf_model.fit(train_processed, epochs=50, 100, 150, 200, 250, validation_data=validation_processed,
                          #  steps_per_epoch=num_iterations)
    history=clf_model.fit_generator(train_generator,steps_per_epoch=train_generator.samples/train_generator.batch_size,epochs=30,validation_data=validation_generator,validation_steps=validation_generator.samples/validation_generator.batch_size,verbose=1,callbacks=[tensorboard_callback])
    # Calculate all relevant metrics
    #model_benchmarks['model_name'].append(mod.el_name)
    #model_benchmarks['num_model_params'].append(pre_trained_model.count_params())
    #model_benchmarks['validation_accuracy'].append(history.history['val_accuracy'][-1])
    loss, acc = clf_model.evaluate_generator(test_generator, steps=245, verbose=0)
    print('using '+str(model_name)+'loss->'+str(loss)+' accuracy->'+str(acc))


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
clf_model.save('pests.h5')

In [ ]:
from keras.layers import GlobalAveragePooling2D, Dense
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import glob

In [ ]:
dataPath = '/content/drive/MyDrive/input'

In [ ]:
# list data path
dirList = glob.glob(dataPath+'/*') # list of all directories in dataPath
dirList.sort() # sorted in alphabetical order
print(dirList)

In [ ]:
def prepare_image(filepath):
    img = cv2.imread(filepath)
    img_resized = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    img_result  = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    return img_result

In [ ]:
num_classes = len(dirList)
labels = [dir.replace(dataPath+"/", "") for dir in dirList]
print(labels)

In [ ]:
def classify_insect(imageFile):
    testData = prepare_image(imageFile).reshape(1,224,224,3)
    testData = testData / 255.0
    predictions = clf_model.predict(testData)
    maxindex = int(np.argmax(predictions))
    print(predictions[0][maxindex],labels[maxindex])
    return labels[maxindex]

In [ ]:
imageFile=dirList[5]+'/5.jpg'
plt.imshow(prepare_image(imageFile))
out = classify_insect(imageFile)


from google.colab import files

with open('result.txt', 'w') as f:
  f.write(out)

files.download('result.txt')